## DEDUPLICATION WITH MINHASH

### Jaccard Similarity
![jaccard](https://media.geeksforgeeks.org/wp-content/uploads/20230811131746/How-to-Calculate-Jaccard-Similarity-in-Python.png)
* SetA = document A
* SetB = document B
* Elements in each set are words
* The more cocument A is alike document B, the more propotion of intersection 

In [148]:
# https://mattilyra.github.io/2017/05/23/document-deduplication-with-lsh.html

s1 = 'สวัสดีครับ'
s2 = 'สวัสดีค่ะ'

shingles1 = set([s1[max(0, i-4):i] for i in range(4, len(s1) + 1)])
shingles2 = set([s2[max(0, i-4):i] for i in range(4, len(s2) + 1)])

len(shingles1 & shingles2) / len(shingles1 | shingles2)

0.4444444444444444

In [149]:
print('what is inside the shingles1')
print( [s1[max(0, i-4):i] for i in range(4, len(s1) + 1)] )

print('shingles1')
print( shingles1 )

what is inside the shingles1
['สวัส', 'วัสด', 'ัสดี', 'สดีค', 'ดีคร', 'ีครั', 'ครับ']
shingles1
{'ีครั', 'ดีคร', 'วัสด', 'ครับ', 'สดีค', 'สวัส', 'ัสดี'}


### MINHASH

In [154]:
import hashlib
import random

class MinHash:
    def __init__(self, num_hashes):
        self.num_hashes = num_hashes
        self.hashes = self._generate_hashes()

    def _generate_hashes(self):
        # Generate random hash functions
        return [(random.randint(1, 1000), random.randint(1, 1000)) for _ in range(self.num_hashes)]

    def hash_set(self, input_set):
        # Initialize with infinite values
        min_hashes = [float('inf')] * self.num_hashes

        for elem in input_set:
            # Hash each element in the set
            for i, (a, b) in enumerate(self.hashes):
                hash_value = (a * hash(elem) + b) % (2**32 - 1)
                min_hashes[i] = min(min_hashes[i], hash_value)

        return min_hashes

    def jaccard_similarity(self, set1, set2):
        # Calculate Jaccard similarity using MinHash
        minhash_set1 = self.hash_set(set1)
        minhash_set2 = self.hash_set(set2)
        
        s1 = set(minhash_set1)
        s2 = set(minhash_set2)
        
        return len(s1 & s2)/len(s1 | s2)


# Example usage:
s1 = 'สวัสดีครับ'
s2 = 'สวัสดีครับบ'

# s1 = 'หนุ่มเปิดกล้องดูลูกสาว จู่ๆ ได้ยินเมียสอนเรียกคนอื่นว่า "พ่อ" สืบจนรู้ความลับใจสลาย · พูดครั้งแรก! · เปิดภาพ 2 ชายต้องสงสัย ซ้อนมอไซค์ฯ มาลักเก๋ง'
# s2 = 'หนุ่มเปิดกล้องดูลูกสาว จู่ๆ ได้ยินภรรยาสอนเรียกคนอื่นว่า "ป๊า" สืบจนรู้ความลับใจสลาย · พูดครั้งแรก! · เปิดภาพ 2 ชายต้องสงสัย ซ้อนมอเตอร์ไซค์ มาลักเก๋ง'
# s2 = 'สืบจนรู้ความลับใจสลาย · พูดครั้งแรก! · เปิดภาพ 2 ชายต้องสงสัย ซ้อนมอเตอร์ไซค์ มาลักเก๋ง หนุ่มเปิดกล้องดูลูกสาว จู่ๆ ได้ยินภรรยาสอนเรียกคนอื่นว่า "ป๊า"'
# s2 = 'Mark Zuckerberg เผยลูกสาวคิดว่าตัวเขาทำงานฟาร์มวัว. 23 ก.พ. 67 · Meizu ยุบแผนพัฒนาสมาร์ทโฟนแบบดั้งเดิม ทุ่มสรรพกำลังทำงานด้าน'

shingles1 = set([s1[max(0, i-4):i] for i in range(4, len(s1) + 1)])
shingles2 = set([s2[max(0, i-4):i] for i in range(4, len(s2) + 1)])

num_hashes = 100
minhash = MinHash(num_hashes)

similarity = minhash.jaccard_similarity(shingles1, shingles2)
print(f"Jaccard Similarity: {similarity}")


Jaccard Similarity: 0.7391304347826086


### MINHASH WITH LIB

In [1]:
!pip install datasketch
!pip install pythainlp


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from datasketch import MinHash, MinHashLSH
from pythainlp.tokenize import word_tokenize

In [3]:
SIMILARITY_THRESHOLD = 0.6
NUM_PERMS = 96
SHINGLE_SIZE = 4

In [4]:
# sentences
s1 = "This is a piece of text"
s2 = "This is a similar piece of text"
s3 = "This is also a similar piece of text"

# init minhash for each sentence
minhash1 = MinHash(num_perm=NUM_PERMS)
minhash2 = MinHash(num_perm=NUM_PERMS)
minhash3 = MinHash(num_perm=NUM_PERMS)

# process minhash for each sentence
for d in set(s1.split()):
    minhash1.update(d.encode("utf8"))
for d in set(s2.split()):
    minhash2.update(d.encode("utf8"))
for d in set(s3.split()):
    minhash3.update(d.encode("utf8"))

In [5]:
# indexing 
lsh = MinHashLSH(threshold=SIMILARITY_THRESHOLD, num_perm=NUM_PERMS)
lsh.insert("text1", minhash1)
lsh.insert("text3", minhash3)

In [6]:
# retrieval
lsh.query(minhash2)

['text1', 'text3']

In [7]:
minhash1.jaccard(minhash2)

0.78125

#### EXMPLE OF DEDUPLICATION

In [8]:
s1 = 'หนุ่มเปิดกล้องดูลูกสาว จู่ๆ ได้ยินเมียสอนเรียกคนอื่นว่า "พ่อ" สืบจนรู้ความลับใจสลาย · พูดครั้งแรก! · เปิดภาพ 2 ชายต้องสงสัย ซ้อนมอไซค์ฯ มาลักเก๋ง'
s2 = 'หนุ่มเปิดกล้องดูลูกสาว จู่ๆ ได้ยินภรรยาสอนเรียกคนอื่นว่า "ป๊า" สืบจนรู้ความลับใจสลาย · พูดครั้งแรก! · เปิดภาพ 2 ชายต้องสงสัย ซ้อนมอเตอร์ไซค์ มาลักเก๋ง'
s3 = 'สืบจนรู้ความลับใจสลาย · พูดครั้งแรก! · เปิดภาพ 2 ชายต้องสงสัย ซ้อนมอไซค์ มาลักเก๋ง หนุ่มเปิดกล้องดูลูกสาว จู่ๆ ได้ยินภรรยาสอนเรียกคนอื่นว่า "ป๊า"'
s4 = 'Mark Zuckerberg เผยลูกสาวคิดว่าตัวเขาทำงานฟาร์มวัว. 23 ก.พ. 67 · Meizu ยุบแผนพัฒนาสมาร์ทโฟนแบบดั้งเดิม ทุ่มสรรพกำลังทำงานด้าน'

minhash1 = MinHash(num_perm=NUM_PERMS)
minhash2 = MinHash(num_perm=NUM_PERMS)
minhash3 = MinHash(num_perm=NUM_PERMS)
minhash4 = MinHash(num_perm=NUM_PERMS)

for d in word_tokenize(s1): minhash1.update(d.encode('utf8'))
for d in word_tokenize(s2): minhash2.update(d.encode('utf8'))
for d in word_tokenize(s3): minhash3.update(d.encode('utf8'))
for d in word_tokenize(s4): minhash4.update(d.encode('utf8'))

In [9]:
lsh = MinHashLSH(threshold=SIMILARITY_THRESHOLD, num_perm=NUM_PERMS)
lsh.insert("text2", minhash2)
lsh.insert("text3", minhash3)
lsh.insert("text4", minhash4)

In [10]:
lsh.query(minhash1)

['text2', 'text3']

In [11]:
minhash1.jaccard(minhash2)

0.8020833333333334

In [12]:
minhash1.jaccard(minhash4)

0.052083333333333336